In [1]:
# !pip install ollama
!pip install numba

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.7 MB 2.2 MB/s eta 0:00:01
   ----------------------- ---------------- 1.6/2.7 MB 2.6 MB/s eta 0:00:01
   ---------------------------------- ----- 2.4/2.7 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   -- ------------------------------------- 1.6/28.1 MB 7.6 MB/s eta 0:00:04
   ---- ----------------------------------- 3.1/28.1 MB 7.7 MB/s eta 0:00:04
   ------- -------------------------------- 5.0/28.1 MB 8.1 MB/s eta 0:00:03
   --------- ------------------------------ 6.6/28.1 MB 8.2 MB/s eta 0:00:03
   ----------- ---------------------------- 8.4/28.1 MB 8.3 MB/s eta 0:00:03
   --------------- ------

In [15]:
import os 
from dotenv import load_dotenv
import ollama

# load all env variables 
load_dotenv()


True

In [2]:
import ollama

response = ollama.chat(
    model="llama3.2",
    messages=[
        {
            "role": "user",
            "content": """Why did Lincoln issue the Emancipation Proclamation?""",
        },
    ],
)
print(response["message"]["content"])

Abraham Lincoln issued the Emancipation Proclamation on January 1, 1863, as a war measure to weaken the Confederacy during the American Civil War. The proclamation declared that all slaves in states still in rebellion against the Union were free, effective immediately.

There were several reasons why Lincoln issued the Emancipation Proclamation:

1. **Military strategy**: Lincoln believed that freeing the slaves would deprive the Confederacy of a significant source of labor and weaken its ability to wage war. By reducing the number of slaves available for work, he hoped to disrupt the Confederate economy and ultimately bring an end to the conflict.
2. **International pressure**: The Emancipation Proclamation was also intended to influence international opinion and put pressure on European powers not to recognize the Confederacy as a legitimate nation. Many in Europe saw slavery as morally reprehensible, and by issuing the proclamation, Lincoln hoped to sway public opinion against the C

## Approach 1:
Let's say we have Q questions, D Datasets, and A answers for each question. We will split P fraction of Q Questions for fine-tuning with golden documents($D_g$: which contains answers to the questions), and the rest (1-P) of Q questions will be trained with distractor documents($D_d$: which are unrelated to the questions). While tuning with distractor documents, model will be forced to learn from the golden documents to answer the 1-P fraction of questions. <b>(Zhang et al.2024)</b>

## Approach 2: (LightRAG)
Use graph structures for text indexing and relevant information retrieval.


In [ ]:
# https://www.youtube.com/watch?v=mE7IDf2SmJg&t=1044s

## For Document Encodeer
# TF-IDF and BM25 
# DrQA Chen et.al 2017
# OrQA lee et.al 2019
# Dens Passage retrieval
# Faiss a vector database
# ColBert
# SPLADE Formal et al. 2021
# DRAGON Lin et al.2023

## For Retrieval of Docs 
# RePlug Shi et.al 2023
# In-Context RALM

## Both retriever and generator
# FiD Izacard & Grave 2020
# KNN-LM (Khandelwal 2019)
# From Document Encoder to Generator
# REALM(Gu et al 2020)
# Atlas()

# Train at scale: TRIME Zhong et al 2023
# Self-RAG (Asai et.al 2023)
#WebGPT
# InstructRetro Wang et.al 2023

In [ ]:
# We will use TriviaQA dataset


### Documnet Encoder

In [11]:
#First we need to do document chunking. There are two ways: For sparse encoding use BM25.
#Sparse Retrieval
# define BM25
import numpy as np
import os
# find frequeency of words in the documents
FOLDER_PATH = 'triviaqa/evidence/web'
# get all files under FOLDER_PATH
data_files = []
for (_,dirs,file_ls) in os.walk(FOLDER_PATH):
    for file in file_ls:
        data_files.append(file)


In [26]:
import numpy as np
from nltk import data as nltk_data

nltk_data.path.append(os.environ["NLTK_DATA"])

file_path='.\\samples\\web\\0\\0_1000687.txt'


# code
rem_specials = lambda w: ''.join([c for c in w if c.isalnum()])
rem_special_words = lambda w: w.isalnum()
vec_rem_special_words = np.vectorize(rem_special_words)
vec_rem_sp = np.vectorize(rem_specials)
file_name = file_path.split("\\")[-1]
# print("Extracting metrics from file: ", file_name)
with open(file_path, encoding="utf-8") as file:
    # Store words and its count
    words={}
    # Read file
    data = file.read()
    # Tokenize entire text into words
    word_ls = np.array(word_tokenize(data)) 
    doclen = len(word_ls)
    print(word_ls[1:4])
    # Remove all non-alnum characters
    word_ls = word_ls[vec_rem_special_words(word_ls)]
    # Remove special characters within the words
    word_ls = vec_rem_sp(word_ls)
    # Count all the unique words in the count
    unique_words, counts = np.unique(word_ls, return_counts=True)
    # Assign each word and its count in the return var
    for index,word in enumerate(unique_words):
        word = word.lower()
        words[word] = int(counts[index])
    print("Extracting finished for: ", file_name)
    print(words)

['.' 'What' 'word']
Extracting finished for:  0_1000687.txt
{'1': 3, '10': 1, '1450': 1, '1855': 2, '1928': 1, '1936': 1, '1960': 1, '3': 2, '4': 2, '5': 1, '6': 1, '7': 2, '8': 2, '9': 1, 'answers': 1, 'alfred': 1, 'anne': 1, 'brigade': 2, 'british': 1, 'brontë': 1, 'carnegie': 1, 'charge': 2, 'chatterley': 1, 'christopher': 1, 'cogito': 1, 'conrad': 1, 'could': 1, 'd': 1, 'dale': 1, 'descartes': 1, 'echo': 1, 'email': 1, 'facebook': 1, 'friends': 1, 'get': 1, 'gutenberg': 1, 'h': 1, 'how': 2, 'i': 2, 'in': 1, 'influence': 1, 'jade': 1, 'korzeniowski': 1, 'lady': 1, 'lawrence': 1, 'light': 2, 'like': 1, 'liverpool': 1, 'lord': 1, 'lover': 1, 'marlowe': 1, 'news': 1, 'novella': 1, 'opinion': 1, 'people': 1, 'rené': 1, 'share': 1, 'subscribe': 1, 'tennyson': 1, 'the': 10, 'twice': 1, 'uk': 1, 'what': 1, 'which': 2, 'who': 6, 'win': 1, 'wright': 1, 'a': 5, 'again': 1, 'am': 1, 'and': 3, 'as': 1, 'ban': 1, 'be': 1, 'between': 2, 'blank': 1, 'book': 4, 'by': 1, 'create': 1, 'daily': 1, 'di

In [ ]:
class s:
    def __init__(self):
        
        